In [15]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import urllib.request
import threading
import time
import requests
import os

In [2]:
animal  =  ["Monkey",  "Elephant",  "cows",
"Cat",  "Dog",  "bear",  "fox",  "Civet", 
"Pangolins", "Rabbit", "Bats", "Whale", 
"Cock", "Owl", "flamingo", "Lizard", "Turtle", 
"Snake", "Frog", "Fish", "shrimp", "Crab", "Snail", 
"Coral", "Jellyfish", "Butterfly", "Flies", "Mosquito", 
"Ants", "Cockroaches", "Spider", "scorpion", "tiger", 
"bird",  "horse", "pig", "Alligator" ,"Alpaca" , 
"Anteater", "donkey", "Bee", "Buffalo", "Camel", 
"Caterpillar", "Cheetah", "Chicken",  "Dragonfly", 
"Duck", "panda", "Giraffe"]

plant = ["Bamboo", "Apple", "Apricot", "Banana", "Bean", 
"Wildflower", "Flower", "Mushroom", "Weed", "Fern" , "Reed", 
"Shrub", "Moss", "Grass", "Palm_tree", "Corn", "Tulip", "Rose",
"Clove", "Dogwood", "Durian", "Ferns", "Fig", "Flax", "Frangipani", 
"Lantana", "Hibiscus", "Bougainvillea", "Pea", "Orchid_Tree", "Rangoon_Creeper",
"Jack_fruit", "Cotton_plant", "Cornelian_tree", "Coffee_plant", "Coconut"
, "wheat", "watermelon", "radish", "carrot"]

furniture = ["bed", "cabinet", "chair", "chests", "clock", 
"desks", "table", "Piano", "Bookcase", "Umbrella", "Clothes", 
"cart", "sofa", "ball", "spoon", "Bowl", "fridge", "pan", "book"]

scenery = ["Cliff", "Bay", "Coast", "Mountains", "Forests", 
"Waterbodies", "Lake", "desert", "farmland", "river", "hedges", 
"plain", "sky", "cave", "cloud", "flower_garden", "glacier", 
"grassland", "horizon", "lighthouse", "plateau", "savannah", "valley", "volcano", "waterfall"]

country = ["USA", "Canada", "France", "Germany", "China", "UK", "Thailand", "Vietnam", "Yemen",
           "Korea", "Mexico", "Japan", "Kazakhstan", "Qatar", "Ukraine", "Russia", "Poland", "Albania",
           "Romania", "Hungary", "Indonesia", "Malaysia", "Singapore", "Afghanistan"]

In [3]:
def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


def get_images(url):
    """
    Returns all image URLs on a single `url`
    """
    soup = BeautifulSoup(urllib.request.urlopen(url), "html.parser")
    urls = []
    for img in soup.find_all("img"):
        img_url = img.attrs.get("data-src")
        
        if not img_url or img_url.endswith(".svg"):
            # if img does not contain src attribute, just skip
            continue
        
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(url, img_url)
        # remove URLs like '/hsts-pixel.gif?c=3.2.5'
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        
        # finally, if the url is valid
        if is_valid(img_url):
            urls.append(img_url)
    print(".", end = ' ')
    return urls

In [4]:
page = 3
query_name = "USA"
sample_url = f"https://www.freepik.com/search?format=search&page={page}&query={query_name}&type=photo"
img_list = get_images(sample_url)
print(img_list)

. ['https://img.freepik.com/free-photo/low-angle-usa-flag-sky_23-2148512980.jpg', 'https://img.freepik.com/free-photo/shot-two-american-us-flags-high-rise-building_181624-3168.jpg', 'https://img.freepik.com/free-photo/painted-red-blue-heart-shape-united-states-american-flag-white-background_23-2148160987.jpg', 'https://img.freepik.com/free-photo/panoramic-shot-amazing-statue-liberty-new-york-city_181624-24247.jpg', 'https://img.freepik.com/free-photo/flapping-usa-flag_23-2148143302.jpg', 'https://img.freepik.com/free-photo/american-flag-wheat-field-representing-strong-agriculture-economy-freedom-united-states-america_342744-597.jpg', 'https://img.freepik.com/free-photo/aerial-view-central-park-manhattan-new-york-city-surrounded-by-skyscrapers_181624-52364.jpg', 'https://img.freepik.com/free-photo/beautiful-morning-view-golden-gate-bridge-san-francisco-usa_181624-30288.jpg', 'https://img.freepik.com/free-photo/skyscrapers-looking-up-sky-modern-metropolis-modern-city_231208-7548.jpg', 'h

In [5]:
class checkTimeThread():
    def __init__(self, nThreads, npage, urltopic):
        self.nThreads = nThreads
        self.ExcuteThread = None
        self.result = []
        self.npage = npage
        self.urltopic = urltopic        

    # Func target
    def main(self, start, end):
        
        for i in range(start,end):
            try:
                self.result.extend(get_images(self.urltopic+str(i)))
            except:
                pass
        
    def __call__(self):
        
        # Create Threads
        threads = []
        
        batch = self.npage//self.nThreads
        for i in range(0, self.npage, batch):
            start = i
            end = i + batch
           
            if end >= self.npage:
                end = self.npage + 1

            threads.append(threading.Thread(target=self.main, args = (start, end)))
        
        start = time.time()
        for i in range(self.nThreads):
            threads[i].start()
        for i in range(self.nThreads):
            threads[i].join()
        end = time.time()
    
        self.ExcuteThread = end - start
    
        return self.result
        
    def __str__(self):
        return f"Excute with {self.nThreads} Thread = {self.ExcuteThread} s\n"

In [13]:
def save_img_url(category_name, category_list, nThreads, npage):
    """
    category_name: Parameter indicating name of the category we want to save.
    category_list: Parameter indicating list of category that we choose from category_name.
    nThreads: number of threads.
    npage: number of pages we want to crawl images from the website.
    """
    try:
        print(f"Created directory {category_name} sucessfully!")
    except OSError:
        print(f"Failed to create directory {category_name}!")
    
    category_folder = f"D:/AI Insights/2021/Practice Projects/Content-based-image-retrieval/crawler/{category_name}"
    if not os.path.exists(category_folder):
        os.makedirs(category_folder, exist_ok=True)
    for name in category_list:
        result = checkTimeThread(min(nThreads, npage//2), npage
                                 ,f"https://www.freepik.com/search?from_query={name}&query={name}&sort=popular&type=photo&page=")()
        result2 = list(set(result))
        print(f"\n{category_name}_{name}.txt contains {len(result2)} images")
        strResult = '\n'.join(result2)
        img_files = f"{category_folder}/{category_name}_{name}.txt"
        with open(img_files, "w") as f:
            f.write(strResult)


In [14]:
nThreads = 30
n_pages = 5

save_img_url("animal", animal, nThreads, n_pages)
save_img_url("country", country, nThreads, n_pages)
save_img_url("furniture", furniture, nThreads, n_pages)
save_img_url("plant", plant, nThreads, n_pages)
save_img_url("scenery", scenery, nThreads, n_pages)

Created directory animal sucessfully!
. . . . 
animal_Monkey.txt contains 146 images
. . . . 
animal_Elephant.txt contains 147 images
. . . . 
animal_cows.txt contains 149 images
. . . . 
animal_Cat.txt contains 149 images
. . . . 
animal_Dog.txt contains 146 images
. . . . 
animal_bear.txt contains 144 images
. . . . 
animal_fox.txt contains 142 images
. . . . 
animal_Civet.txt contains 150 images
. . . . 
animal_Pangolins.txt contains 9 images
. . . . 
animal_Rabbit.txt contains 145 images
. . . . 
animal_Bats.txt contains 147 images
. . . . 
animal_Whale.txt contains 145 images
. . . . 
animal_Cock.txt contains 139 images
. . . . 
animal_Owl.txt contains 149 images
. . . . 
animal_flamingo.txt contains 146 images
. . . . 
animal_Lizard.txt contains 150 images
. . . . 
animal_Turtle.txt contains 146 images
. . . . 
animal_Snake.txt contains 148 images
. . . . 
animal_Frog.txt contains 150 images
. . . . 
animal_Fish.txt contains 146 images
. . . . 
animal_shrimp.txt contains 150 imag

In [16]:
class DownloadFromUrl():
    def __init__(self, nThreads, urls, destination_folder):
        self.nThreads = nThreads
        self.urls = urls
        self.amount = len(urls)
        self.destination_folder = destination_folder
        
    def get_url_status(self, url):  # checks status for each url in list urls
        try:
            urllib.request.urlopen(url)
            requests.get(url)
        except Exception as e:
            return False
        else:
            return True
        
    # Func target
    def download_image(self, start, end):

        for i in range(start, end):
            try:
                response=requests.get(self.urls[i])
            except Exception as e:
                continue
            with open(f"{self.destination_folder}/{i}.jpg", "wb") as f:
                f.write(response.content)
            print('.', end=" ")         
                 
    def __call__(self):
        try:
            threads = []
            batch = self.n//self.nThreads
            for i in range(0, self.n, batch):
                start = i
                end = i + batch

                if end >= self.amount:
                    end = self.amount + 1

                threads.append(threading.Thread(target=self.download_image, args = (start, end)))
        
            for i in range(self.nThreads):
                threads[i].start()
            for i in range(self.nThreads):
                threads[i].join()

        except Exception as e:
            pass   

In [17]:
l = []
for name_txt in os.listdir():
    if name_txt.endswith(".txt"):
        l.append(name_txt)